<a href="https://colab.research.google.com/github/isold23/RAG/blob/main/search_from_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import sys

template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

print("sys.path[0] = ", sys.path[0])
print(os.listdir("/content/drive/MyDrive"))
os.chdir("/content/drive/MyDrive")
print("sys.path[0] = ", sys.path[0])


llm = LlamaCpp(
    model_path="/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q2_K.bin",
    #model_path="/drive/MyDrive/llama-2-13b-chat.ggmlv3.q4_0.bin",
    n_ctx=6000,
    n_gpu_layers=512,
    n_batch=30,
    callback_manager=callback_manager,
    temperature = 0.9,
    max_tokens = 4095,
    n_parts=1,

)

llm_chain = LLMChain(prompt=prompt, llm=llm)


#loader = UnstructuredFileLoader("AI discovers over 20K taxable French swimming pools.docx")
loader = UnstructuredFileLoader("/content/drive/MyDrive/lsmtree.pdf")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

db = FAISS.from_documents(docs, hf_embedding)

# save embeddings in local directory
db.save_local("faiss_AiArticle")

# load from local
db = FAISS.load_local("faiss_AiArticle/", embeddings=hf_embedding)

#query = "why french government using AI?"
query = "Introducing lsm tree"
search = db.similarity_search(query, k=2)
print("---------------------")
print(search)

template = '''Context: {context}

Based on Context provide me answer for following question
Question: {question}

Tell me the information about the fact. The answer should be from context only
do not use general knowledge to answer the query'''

prompt = PromptTemplate(input_variables=["context", "question"], template= template)
final_prompt = prompt.format(question=query, context=search)

llm_chain.run(final_prompt)



sys.path[0] =  /content
['llama-2-7b-chat.ggmlv3.q2_K.bin', 'lsmtree.pdf', 'Colab Notebooks', 'faiss_AiArticle', 'Wav2Lip', 'Google AI Studio']
sys.path[0] =  /content


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [41]:
!pip install llama-cpp-python langchain faiss-gpu sentence-transformers InstructorEmbedding "unstructured[pdf]"

In [34]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
